<p align ="center">
AME 5763
</p>
<p align ="center">
Final Exam
</p>
<p align="center">
Blake T Johnson
</p>
<p align="center">
December 2, 2024
</p>

In [104]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp
from IPython.core.display import HTML
from sympy import Matrix
import matplotlib.tri as tri

# Problem 1

Derive the finite element model of the equation

$$
-\frac{d}{dx} \left[ (1 + x) \frac{du}{dx} \right] = f, \quad 0 < x < 1
$$

for the boundary conditions of the type

$$
u(0) = \bar{u}, \quad \left[ (1 + x) \frac{du}{dx} \right]_{x=1} = \bar{q}
$$

and solve the problem for the data $ f = 0 $, $\bar{u} = 1 $, and $ \bar{q} = 0 $. Use two linear finite elements.


In [105]:
'''
The problem gives us the strong form of the equation. To begin I set up variables for each of the given parameters.
'''

# set x and u for as symbols for deriving equations
x = sp.symbols('x')
u = sp.Function('u')(x)

# Define the equation parameters
f = 0  # given as 0
ubar = 1  # boundary condition at x=0
qbar = 0  # boundary condition at x=1

# Differential equation
diff_eq = -sp.diff((1 + x) * sp.diff(u, x), x) - f
# Print the equation
print("Differential Equation:", diff_eq)


Differential Equation: -(x + 1)*Derivative(u(x), (x, 2)) - Derivative(u(x), x)


To calculate the weak form, multiply the governing equation by the weight function and integrate:

$$
\int_{0}^{1} w(x) \left[- \frac{d}{dx}\left((x + 1) \frac{du}{dx}\right)\right] dx = 0
$$

$$
- \int_{0}^{1}  \frac{d}{dx} \left[ w(x) \left((x + 1) \frac{du}{dx}\right)\right] dx = 0
$$

Let $ v = (x + 1) \frac{du}{dx} $

$$
- \int_{0}^{1}  \frac{d}{dx} \left[ w(x) v \right] dx = 0
$$

Note:
$$
\frac{d}{dx}(w \cdot v) = w \frac{dv}{dx} + v \frac{dw}{dx}
$$

$$
w \frac{dv}{dx} = \frac{d}{dx}[w \cdot v] - v \frac{dw}{dx}
$$

Applying this to my equation:

$$
\int_{0}^{1} w(x) \left[ \frac{d}{dx} \left( (x+1) \frac{du}{dx} \right) \right] dx = \int_{0}^{1} \frac{d}{dx}\left[ w(x) (x + 1) \frac{du}{dx}\right] dx - \int_{0}^{1} (x + 1) \frac{du}{dx} \frac{dw}{dx} dx
$$

$$
\int_{0}^{1} w(x) \left[ \frac{d}{dx} \left( (x+1) \frac{du}{dx} \right) \right] dx = \left[ w(x) (x + 1) \frac{du}{dx} \right]_{x=0}^{1} - \int_{0}^{1} (x + 1) \frac{du}{dx} \frac{dw}{dx} dx
$$

The weight funciton at x = 0 is 0. So the First term will dissapear at x = 0.

$$
\int_{0}^{1} w(x) \left[ \frac{d}{dx} \left( (x+1) \frac{du}{dx} \right) \right] dx = \left[ w(x) (x + 1) \frac{du}{dx} \right]_{x=1} - \int_{0}^{1} (x + 1) \frac{du}{dx} \frac{dw}{dx} dx
$$

$$
\int_{0}^{1} w(x) \left[ \frac{d}{dx} \left( (x+1) \frac{du}{dx} \right) \right] dx = 2 \left[ w(x) \frac{du}{dx} \right]_{x=1} - \int_{0}^{1} (x + 1) \frac{du}{dx} \frac{dw}{dx} dx
$$


In [106]:
"""
Next I ran the above equation through python to get the weak form
"""

# Define variables and functions
x = sp.Symbol('x')                  # Spatial variable
u = sp.Function('u')(x)             # Trial function
w = sp.Function('w')(x)             # Test function
f = sp.Function('f')(x)             # Source term, now a function of x

# Define the weak form
du_dx = sp.diff(u, x)               # Derivative of u
dw_dx = sp.diff(w, x)               # Derivative of w
weak_form = sp.integrate((1 + x) * du_dx * dw_dx, (x, 0, 1)) - sp.integrate(w * f, (x, 0, 1))

# Integration by parts to handle the first term
boundary_term = w * (1 + x) * du_dx    # Boundary term after integration by parts
boundary_contrib = boundary_term.subs(x, 1) - boundary_term.subs(x, 0)

# Reformulate the weak form with boundary contributions
weak_form = boundary_contrib - sp.integrate(dw_dx * (1 + x) * du_dx, (x, 0, 1))

'''
The weight function is zero at x = 0 due to the Dirichlet boundary condition.
w(x) = 0 at x = 0.
So I simplified the weak form to get my final weak form.
'''

# Simplified weak form with boundary term at x=1
boundary_term_simplified = boundary_term.subs(x, 1)  # Only keep x=1 contribution
weak_form_simplified = boundary_term_simplified - sp.integrate(dw_dx * (1 + x) * du_dx, (x, 0, 1))

# Display results
print("Simplified Weak Form:")
sp.pretty_print(weak_form_simplified)

Simplified Weak Form:
                        1                             
                        ⌠                             
       ⎛d       ⎞│      ⎮         d        d          
2⋅w(1)⋅⎜──(u(x))⎟│    - ⎮ (x + 1)⋅──(u(x))⋅──(w(x)) dx
       ⎝dx      ⎠│x=1   ⎮         dx       dx         
                        ⌡                             
                        0                             


In [107]:
'''
Now that I have the weak form I followed chapter 4 to construct 
two linear element shape functions and the corresponding shape function

Since they are linear and I need two, I decided to use two of the same shape functions
element 1: [0,0.5]
element 2: [0.5,1]
I used the equations from pg 83 for the shape functions
'''

# Define the variable and nodes
x = sp.Symbol('x')
x1, x2, x3 = 0, 0.5, 1  # Nodes for the two-element mesh

# Local shape functions for Element 1 (0 to 0.5)
N1 = (x - x2) / (x1 - x2)  # Shape function for Node 1 (updated convention)
N2 = (x - x1) / (x2 - x1)  # Shape function for Node 2 (unchanged)

# Local shape functions for Element 2 (0.5 to 1)
#N1_e2 = (x - x3) / (x2 - x3)  # Shape function for Node 2 (shared, updated convention)
#N2_e2 = (x - x2) / (x3 - x2)  # Shape function for Node 3 (unchanged)

# Display the shape functions
print("Shape Functions for Element 1:")
sp.pretty_print(N1)
sp.pretty_print(N2)

#print("\nShape Functions for Element 2:")
#sp.pretty_print(N1_e2)
#sp.pretty_print(N2_e2)

Shape Functions for Element 1:
1.0 - 2.0⋅x
2.0⋅x


In [108]:
'''
Next I can set up the B matrix for the two elements
'''

# Shape function derivatives
dN1_dx = sp.diff(N1, x)
dN2_dx = sp.diff(N2, x)

# B-matrix (derivatives of shape functions)
B = sp.Matrix([dN1_dx, dN2_dx])

print("\nB-Matrix:")
sp.pretty_print(B)




B-Matrix:
⎡-2.0⎤
⎢    ⎥
⎣2.0 ⎦


In [109]:
'''
Now I can set up the K value for the two elements and the global K. K = B^T * Ak * B
'''
x, x1, x2 = sp.symbols('x x1 x2')

# Stiffness matrix Ke for one element
Ke = sp.Matrix(2, 2, lambda i, j: sp.integrate(B[i] * (1 + x) * B[j], (x, x1, x2)))

# Simplify the generic stiffness matrix
Ke_simplified = sp.simplify(Ke)

# Display the simplified stiffness matrix
print("Simplified Element Stiffness Matrix (Ke):")
sp.pprint(Ke_simplified)

# Substitute values for Element 1: [0, 0.5]
Ke_e1 = Ke.subs({x1: 0, x2: 0.5})
print("\nElement Stiffness Matrix for Element 1 (Ke_e1):")
sp.pprint(Ke_e1)

# Substitute values for Element 2: [0.5, 1]
Ke_e2 = Ke.subs({x1: 0.5, x2: 1})
print("\nElement Stiffness Matrix for Element 2 (Ke_e2):")
sp.pprint(Ke_e2)

Simplified Element Stiffness Matrix (Ke):
⎡        2                  2                  2                  2          ⎤
⎢- 2.0⋅x₁  - 4.0⋅x₁ + 2.0⋅x₂  + 4.0⋅x₂   2.0⋅x₁  + 4.0⋅x₁ - 2.0⋅x₂  - 4.0⋅x₂ ⎥
⎢                                                                            ⎥
⎢       2                  2                    2                  2         ⎥
⎣ 2.0⋅x₁  + 4.0⋅x₁ - 2.0⋅x₂  - 4.0⋅x₂   - 2.0⋅x₁  - 4.0⋅x₁ + 2.0⋅x₂  + 4.0⋅x₂⎦

Element Stiffness Matrix for Element 1 (Ke_e1):
⎡2.5   -2.5⎤
⎢          ⎥
⎣-2.5  2.5 ⎦

Element Stiffness Matrix for Element 2 (Ke_e2):
⎡3.5   -3.5⎤
⎢          ⎥
⎣-3.5  3.5 ⎦


In [110]:
'''
Assemble the global stiffness matrix numerically. This is a 3x3 with K1 being the first 2x2 and K2 being the last 2x2
'''

# Set up a matrix of zeros for the global stiffness matrix
K_global = sp.zeros(3, 3)

# Add contributions from Element 1
K_global[0:2, 0:2] += Ke_e1

# Add contributions from Element 2
K_global[1:3, 1:3] += Ke_e2

print("\nGlobal Stiffness Matrix (K_global):")
sp.pprint(K_global)

'''
This matrix does not take into consideration the boundary conditions.
The determinant = 0 so I need to adjust the boundaries correctly
'''

# Apply Dirichlet boundary condition at x = 0 (u(0) = 1)
K_global[0, :] = sp.zeros(1, 3)  # Zero out the first row
K_global[:, 0] = sp.zeros(3, 1)  # Zero out the first column
K_global[0, 0] = 1  # Set diagonal entry to 1

# The new global stiffness matrix after applying Dirichlet boundary condition
print("\nModified Global Stiffness Matrix (K_global):")
sp.pprint(K_global)



Global Stiffness Matrix (K_global):
⎡2.5   -2.5   0  ⎤
⎢                ⎥
⎢-2.5  6.0   -3.5⎥
⎢                ⎥
⎣ 0    -3.5  3.5 ⎦

Modified Global Stiffness Matrix (K_global):
⎡1   0     0  ⎤
⎢             ⎥
⎢0  6.0   -3.5⎥
⎢             ⎥
⎣0  -3.5  3.5 ⎦


In [111]:
'''
Finally I can solve for the displacement vector using the global stiffness matrix and the external force vector
Using Kd = F I can solve for the displacement vector
using d = K^-1 * F
'''
# Apply Dirichlet boundary condition at x = 0, u(0) = 1, and q = 0

f = sp.Matrix([1, 0, 0])  # External force vector


print("\n External Force Vector (F):")
sp.pprint(f)

d = K_global.inv() * f  # Correct matrix multiplication order
print("\nDisplacement Vector (d):")
sp.pprint(d)


 External Force Vector (F):
⎡1⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

Displacement Vector (d):
⎡1⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦


My final results show that there is displacement of node 1 by 1 and no other displacement. Phyisically this is a little confusing but also makes sense as there is no external forcce applied to the system and the given conditions state that there is a force of 1 at ubar.

# Problem 2

Using the linear field $\phi = \phi(x, y, z) = a_{1} + a_{2}x + a_{3}y + a_{4}z$, prove that isoparametric elements have the ability to represent constant gradients of the field.

## Step 1: Field and Shape Functions

Starting with the linear field:
$$
\phi(x, y, z) = a_1 + a_2x + a_3y + a_4z.
$$

$ \phi $ can be interpolated using the shape functions $ N_i $ and the nodal values $ \phi_i $:
$$
\phi(x, y, z) = \sum_{i=1}^n N_i(x, y, z) \phi_i.
$$

The shape functions $ N_i $ are linear functions of $ x, y, z $:
$$
N_i(x, y, z) = a_i + b_i x + c_i y + d_i z,
$$

Then the field becomes:
$$
\phi(x, y, z) = \sum_{i=1}^n \left( a_i + b_i x + c_i y + d_i z \right) \phi_i.
$$

$$
\phi(x, y, z) = \sum_{i=1}^n \left( a_i \phi_i + b_i x \phi_i + c_i y \phi_i + d_i z \phi_i \right).
$$


$$
\phi(x, y, z) = \sum_{i=1}^n a_i \phi_i + \sum_{i=1}^n b_i \phi_i x + \sum_{i=1}^n c_i \phi_i y + \sum_{i=1}^n d_i \phi_i z.
$$


$$
\phi(x, y, z) = \left( \sum_{i=1}^n a_i \phi_i \right) + \left( \sum_{i=1}^n b_i \phi_i \right) x + \left( \sum_{i=1}^n c_i \phi_i \right) y + \left( \sum_{i=1}^n d_i \phi_i \right) z.
$$

This equation matches the given form of $ \phi(x, y, z) = a_1 + a_2x + a_3y + a_4z $, with:
$$
a_1 = \sum_{i=1}^n a_i \phi_i, \quad
a_2 = \sum_{i=1}^n b_i \phi_i, \quad
a_3 = \sum_{i=1}^n c_i \phi_i, \quad
a_4 = \sum_{i=1}^n d_i \phi_i.
$$

Thus, $ \phi(x, y, z) $ is linear because:
- The shape functions $ N_i(x, y, z) $ are linear,
- The coefficients $ a_i, b_i, c_i, d_i $ are constants, and
- The nodal values $ \phi_i $ are constants.


## Step 2: Gradients in Natural Coordinates

The gradient is defined in 7.4.2 of the text as 
$$
\nabla \phi^e = B^e d^e
$$

Since the textbook uses the four node quadralateral element as an example, I am using that as an example for this problem.



From **Step 1**, the field is expressed as:
$$
\phi(\xi, \eta, \zeta) = \sum_{i=1}^n N_i(\xi, \eta, \zeta) \phi_i.
$$

### 1. Gradient of $\phi $ in Natural Coordinates
The gradient of $ \phi $ in natural coordinates is given by the Jacobian (page 167):
$$
\begin{bmatrix}
\frac{\partial \phi}{\partial \xi} \\
\frac{\partial \phi}{\partial \eta} \\
\frac{\partial \phi}{\partial \zeta}
\end{bmatrix}
=
\begin{bmatrix}
\sum_{i=1}^n \frac{\partial N_i}{\partial \xi} \phi_i \\
\sum_{i=1}^n \frac{\partial N_i}{\partial \eta} \phi_i \\
\sum_{i=1}^n \frac{\partial N_i}{\partial \zeta} \phi_i
\end{bmatrix}.
$$

### 2. Derivatives of Shape Functions
The shape functions $ N_i $ are **linear functions** of $ \xi, \eta, \zeta $, such as:
$$
N_i(\xi, \eta, \zeta) = a_i + b_i \xi + c_i \eta + d_i \zeta.
$$

The derivatives with respect to $ \xi, \eta, \zeta $ are therefore constants:
$$
\frac{\partial N_i}{\partial \xi} = b_i, \quad
\frac{\partial N_i}{\partial \eta} = c_i, \quad
\frac{\partial N_i}{\partial \zeta} = d_i.
$$

Substituting into the gradient expression, we find:
$$
\frac{\partial \phi}{\partial \xi} = \sum_{i=1}^n b_i \phi_i, \quad
\frac{\partial \phi}{\partial \eta} = \sum_{i=1}^n c_i \phi_i, \quad
\frac{\partial \phi}{\partial \zeta} = \sum_{i=1}^n d_i \phi_i.
$$

These are constant values since $ b_i, c_i, d_i, \phi_i $ are constants. (We are considering the field $\phi$ at a specific node $\phi_i$ which is why it can be considered a constant).
Thus, the gradient of $ \phi $ in natural coordinates is constant.

## Step 3: Transformation to Physical Coordinates
Next I want to convert from the physical coordinates to the natural coordinates.

### 1. Chain Rule Transformation
The gradient of $ \phi $ in physical coordinates ($ x, y, z $) is related to its gradient in natural coordinates ($ \xi, \eta, \zeta $) by the chain rule (From page 167 I took the inverse of he Jacobian here to get it in a format fro the chain rule):

$$
\begin{bmatrix}
\frac{\partial \phi}{\partial x} \\
\frac{\partial \phi}{\partial y} \\
\frac{\partial \phi}{\partial z}
\end{bmatrix}
=
\begin{bmatrix}
\frac{\partial \xi}{\partial x} & \frac{\partial \eta}{\partial x} & \frac{\partial \zeta}{\partial x} \\
\frac{\partial \xi}{\partial y} & \frac{\partial \eta}{\partial y} & \frac{\partial \zeta}{\partial y} \\
\frac{\partial \xi}{\partial z} & \frac{\partial \eta}{\partial z} & \frac{\partial \zeta}{\partial z}
\end{bmatrix}
\begin{bmatrix}
\frac{\partial \phi}{\partial \xi} \\
\frac{\partial \phi}{\partial \eta} \\
\frac{\partial \phi}{\partial \zeta}
\end{bmatrix}.
$$

Here:
- The matrix on the left is the gradient in physical coordinates ($ \nabla \phi $ in $ x, y, z $).
- The matrix in the middle is the Jacobian matrix inverse, which transforms derivatives from natural to physical coordinates.
- The matrix on the right is the gradient in natural coordinates ($ \nabla \phi $ in $ \xi, \eta, \zeta $).

### 2. Jacobian Matrix
The Jacobian matrix $ J $ relates the physical coordinates ($ x, y, z $) to the natural coordinates ($ \xi, \eta, \zeta $):
$$
J =
\begin{bmatrix}
\frac{\partial x}{\partial \xi} & \frac{\partial x}{\partial \eta} & \frac{\partial x}{\partial \zeta} \\
\frac{\partial y}{\partial \xi} & \frac{\partial y}{\partial \eta} & \frac{\partial y}{\partial \zeta} \\
\frac{\partial z}{\partial \xi} & \frac{\partial z}{\partial \eta} & \frac{\partial z}{\partial \zeta}
\end{bmatrix}.
$$

The inverse of this matrix ($ J^{-1} $) is what is above in the chain rule transformation.

### 3. Constant Gradient in Physical Coordinates
From **Step 2**, we know that $ \nabla \phi $ in natural coordinates ($ \xi, \eta, \zeta $) is constant:
$$
\begin{bmatrix}
\frac{\partial \phi}{\partial \xi} \\
\frac{\partial \phi}{\partial \eta} \\
\frac{\partial \phi}{\partial \zeta}
\end{bmatrix}
=
\text{constant vector}.
$$

The Jacobian $ J $ is a constant matrix for linear elements (since the shape functions $ N_i $ are linear). Therefore:
$$
\nabla \phi \text{ in physical coordinates } (x, y, z) = J^{-1} \cdot \nabla \phi \text{ in natural coordinates } (\xi, \eta, \zeta)
$$
is also constant.


# Problem 3

Develop the weak form for the following partial differential equation:

$$
- \frac{\partial}{\partial x} \left( a_1 \frac{\partial u}{\partial x} \right) - \frac{\partial}{\partial y} \left( a_2 \frac{\partial u}{\partial y} \right) + a_0 u = f
$$

in a two-dimensional domain $R$ with boundary conditions: $u = \bar{u}$ on $S_1$ and 

$$
a_1 \frac{\partial u}{\partial x} n_x + a_2 \frac{\partial u}{\partial y} n_y = \bar{g} \text{ on } S_2
$$ 

where $n_x$ and $n_y$ are the $x$ and $y$ components of the normal to the surface $S_2$, $\mathbf{n}$. Here $a_0, a_1, a_2, f, \bar{u}$ and $\bar{g}$ are known functions of $x$ and $y$ in $R$.


### Step 1: List the values given in the problem

The given PDE is:

$$
- \frac{\partial}{\partial x} \left( a_1 \frac{\partial u}{\partial x} \right) - \frac{\partial}{\partial y} \left( a_2 \frac{\partial u}{\partial y} \right) + a_0 u = f \quad \text{in domain } R
$$

- **Coefficients:**
  - $a_1, a_2$: Functions of $x$ and $y$, represent scaling factors for the derivatives of $u$ with respect to $x$ and $y$.
  - $a_0$: A function of $x$ and $y$, represents a source/sink term.
- **Dependent Variable:**
  - $u$: The solution we are trying to find.
- **Source Term:**
  - $f$: Represents the known forcing function.

**Boundary Conditions:**

1. **Dirichlet Condition** on boundary $S_1$:

$$
u = \bar{u} \quad \text{on } S_1
$$

2. **Neumann Condition** on boundary $S_2$:

$$
a_1 \frac{\partial u}{\partial x} n_x + a_2 \frac{\partial u}{\partial y} n_y = \bar{g} \quad \text{on } S_2
$$

- $n_x$ and $n_y$ are the components of the normal vector to the boundary $S_2$.
- $\bar{g}$ represents the flux on the boundary.

The PDE is defined over a two-dimensional domain $R$,

### Step 2: Introducing the Test Function

**Strong Form of the PDE:**

$$
- \frac{\partial}{\partial x} \left( a_1 \frac{\partial u}{\partial x} \right) - \frac{\partial}{\partial y} \left( a_2 \frac{\partial u}{\partial y} \right) + a_0 u = f \quad \text{in domain } R
$$

with boundary conditions:

- **Dirichlet Condition** on $S_1$:
  
  $$
  u = \bar{u} \quad \text{on } S_1
  $$

- **Neumann Condition** on $S_2$:

  $$
  a_1 \frac{\partial u}{\partial x} n_x + a_2 \frac{\partial u}{\partial y} n_y = \bar{g} \quad \text{on } S_2
  $$

**Introducing the Test Function $w(x)$:**

- Test function $w(x)$ is zero on the Dirichlet boundary ($S_1$) and arbitrary in the interior of the domain $R$.
- To derive the weak form, multiply the PDE by $w(x)$ and integrate over the domain $R$.

**Formulation:**

$$
\int_R w(x) \left( - \frac{\partial}{\partial x} \left( a_1 \frac{\partial u}{\partial x} \right) - \frac{\partial}{\partial y} \left( a_2 \frac{\partial u}{\partial y} \right) + a_0 u \right) \, dA = \int_R w(x) f \, dA
$$

$$
\int_R \left( - w(x) \frac{\partial}{\partial x} \left( a_1 \frac{\partial u}{\partial x} \right) \right) dx  - \int_y \left( w(y) \frac{\partial}{\partial y} \left( a_2 \frac{\partial u}{\partial y} \right) \right) dy  +  \int_x \int_y w(x,y) a_0 \ u \ dy dx = \int_R w(x) f \, dA
$$


### Step 3: Integrate by Parts
***Term 1:***

$$
- \frac{\partial}{\partial x} \left(a_1 \frac{\partial u}{\partial x}\right)
$$

Using product rule:
$$
- \frac{\partial}{\partial x} \left(a_1 \frac{\partial u}{\partial x}\right) = -a_1 \frac{\partial^{2}u}{\partial x} - \frac{\partial a_1}{\partial x} \frac{\partial u}{\partial x}
$$

Now I can multiply by the weight function and integrate

$$
- \int_x w(x) \frac{\partial}{\partial x} \left(a_1 \frac{\partial u}{\partial x} \right) = \int_x w(x) \left[ -a_1 \frac{\partial^{2} u }{\partial x} - \frac{\partial a_1}{\partial x} \frac{\partial u}{\partial x} \right]
$$

$$
 \int_x w(x) \left[ -a_1 \frac{\partial^{2} u }{\partial x} - \frac{\partial a_1}{\partial x} \frac{\partial u}{\partial x} \right]
= - \int_x w(x) a_1 \frac{\partial^{2} u }{\partial x} dx - \int_x w(x) \frac{\partial a_1}{\partial x} \frac{\partial u}{\partial x} dx
$$

Using integration by parts on $- \int_x w(x) a_1 \frac{\partial^{2} u }{\partial x} dx$:
$$
- \int_x w(x) a_1 \frac{\partial^{2} u }{\partial x} dx
$$

$$
u = w(x)a_1 \\
du = w(x) \frac{\partial a_1}{\partial x} + a_1 \frac{\partial w(x)}{\partial x} \\
v = \frac{ \partial u}{\partial x}\\
dv = \frac{\partial^{2} u}{\partial x}
$$

$$
\int u dv = uv - \int v du\\
- \int_x w(x) a_1 \frac{\partial^{2} u }{\partial x} dx = - \left[w(x) a_1 \frac{\partial u}{\partial x} \Big|_{boundary} - \int_x \frac{\partial u}{\partial x} \left( w(x) \frac{\partial a_1}{\partial x} + a_1 \frac{\partial w(x)}{\partial x} \right) dx \right]\\
- \int_x w(x) a_1 \frac{\partial^{2} u }{\partial x} dx = - \left[w(x) a_1 \frac{\partial u}{\partial x}\big|_{boundary} - \int_x w(x) \frac{\partial u}{\partial x} \frac{\partial a_1}{\partial x} dx + \int_x  a_1 \frac{\partial w(x)}{\partial x} \frac{\partial u}{\partial  x} dx \right]
$$

Now I can substitute this back into the original integral:
$$
- \int_x w(x) a_1 \frac{\partial^2{u}}{\partial x}dx - \int_x w(x) \frac{\partial a_1}{\partial x} \frac{\partial u}{\partial x}dx
= - \left[w(x) a_1 \frac{\partial u}{\partial x} \big|_{boundary} - \int_x w(x) \frac{\partial u}{\partial x} \frac{\partial a_1}{\partial x} dx + \int_x  a_1 \frac{\partial w(x)}{\partial x} \frac{\partial u}{\partial  x} dx \right] - \int_x w(x) \frac{\partial a_1}{\partial x} \frac{\partial u}{\partial x}dx
$$

This simplifies to:
$$
- \int_x w(x) a_1 \frac{\partial^2{u}}{\partial x}dx - \int_x w(x) \frac{\partial a_1}{\partial x} \frac{\partial u}{\partial x}dx
= - \left[w(x) a_1 \frac{\partial u}{\partial x}\Big|_{boundary} + \int_x  a_1 \frac{\partial w(x)}{\partial x} \frac{\partial u}{\partial  x} dx \right] \\

- \int_x w(x) a_1 \frac{\partial^2{u}}{\partial x}dx - \int_x w(x) \frac{\partial a_1}{\partial x} \frac{\partial u}{\partial x}dx
= - w(x) a_1 \frac{\partial u}{\partial x} \Big|_{boundary} - \int_x  a_1 \frac{\partial w(x)}{\partial x} \frac{\partial u}{\partial  x} dx
$$

***Boundary Conditions***

$ -w(x) a_1 \frac{\partial u}{\partial x} $ is a boundary term which evaluates the flux (in this case) at the boundary of the domain. On $S_1$ the weight function $= 0$ and goes away. on $S_2$ the boundary term is $a_1 \frac{\partial u}{\partial x} n_x + a_2 \frac{\partial u}{\partial y} n_y = \bar{y}$

If we apply this boundary to the integrated form of the boundary term we get:
$$
\int_{\partial S_2} w(x) a_{1} \frac{\partial u}{\partial x}n_x dS
$$

And the final form of the First Term is:
$$
- \int_x w(x) \frac{\partial}{\partial x} \left(a_1 \frac{\partial u}{\partial x} \right) = - \int_{\partial S_2} w(x) a_{1} \frac{\partial u}{\partial x}n_x dS - \int_x  a_1 \frac{\partial w(x)}{\partial x} \frac{\partial u}{\partial  x} dx
$$

***Term 2:***

The second term can be treated the same as the first term only replacing $a_1$ with $a_2$ and $x$ for $y$ we get:
$$
- \int_y w(y) \frac{\partial }{\partial y} \left[ a_2 \frac{\partial u}{\partial y}\right] dx = - \int_{\partial S_2} w(y) a_{2} \frac{\partial u}{\partial y}n_y dS - \int_y a_2 \frac{\partial w(y)}{\partial y}\frac{\partial u}{\partial y} dy
$$

*** Term 3: ***
The third term represents a volume integral, so it involves both x and y. This suggests a double integral is needed.
$$
\int_x \int_y w(x,y) a_{0}(x,y)u(x,y) dy dx
$$


### Step 4 Combining the Terms to get the weak form:
Next I combine the Boundary Conditions Terms together and I also combine the Volume Integral terms into one term.
$$
\int_R w(x) \left( - \frac{\partial}{\partial x} \left( a_1 \frac{\partial u}{\partial x} \right) - \frac{\partial}{\partial y} \left( a_2 \frac{\partial u}{\partial y} \right) + a_0 u \right) \, dA = \\  - \int_{\partial S_2} w(x,y) a_{1} \frac{\partial u}{\partial x}n_x dS
- \int_x  a_1 \frac{\partial w(x)}{\partial x} \frac{\partial u}{\partial  x} dx - \int_{\partial R} w(x,y) a_{2} \frac{\partial u}{\partial y}n_y dS - \int_y a_2 \frac{\partial w(y)}{\partial y}\frac{\partial u}{\partial y} dy + \int_x \int_y w(x,y) a_{0}(x,y)u(x,y) dy dx
$$

$$
\int_R w(x,y) \left( - \frac{\partial}{\partial x} \left( a_1 \frac{\partial u}{\partial x} \right) - \frac{\partial}{\partial y} \left( a_2 \frac{\partial u}{\partial y} \right) + a_0 u \right) \, dA = \\  - \int_{\partial S_2} w(x,y) a_{1} \frac{\partial u}{\partial x}n_x dS - \int_{\partial R} w(x,y) a_{2} \frac{\partial u}{\partial y}n_y dS 
- \int_R  a_1 \frac{\partial w(x)}{\partial x} \frac{\partial u}{\partial  x} dR  - \int_R a_2 \frac{\partial w(y)}{\partial y}\frac{\partial u}{\partial y} dR + \int_R w(x,y) a_{0}(x,y)u(x,y) dR
$$

$$
\int_R w(x,y) \left( - \frac{\partial}{\partial x} \left( a_1 \frac{\partial u}{\partial x} \right) - \frac{\partial}{\partial y} \left( a_2 \frac{\partial u}{\partial y} \right) + a_0 u \right) \, dA = \\  - \int_{\partial S_2} w(x,y) \left( a_{1} \frac{\partial u}{\partial x}n_x + a_{2} \frac{\partial u}{\partial y}n_y \right) dS 
  - \int_R  \left( a_1 \frac{\partial w(x)}{\partial x} \frac{\partial u}{\partial  x} + a_2 \frac{\partial w(y)}{\partial y}\frac{\partial u}{\partial y}  +  w(x,y) a_{0}(x,y)u(x,y) \right) dR
$$

$$
\int_R w(x,y) \left( - \frac{\partial}{\partial x} \left( a_1 \frac{\partial u}{\partial x} \right) - \frac{\partial}{\partial y} \left( a_2 \frac{\partial u}{\partial y} \right) + a_0 u \right) \, dA = \\  - \int_{\partial S_2} w(x,y) \left( \bar{y} \right) dS
- \int_R  \left( a_1 \frac{\partial w(x)}{\partial x} \frac{\partial u}{\partial  x} + a_2 \frac{\partial w(y)}{\partial y}\frac{\partial u}{\partial y} +  w(x,y) a_{0}(x,y)u(x,y)\right) dR
$$

### Final Form:

$$
 \int_R  \left( a_1 \frac{\partial w}{\partial x} \frac{\partial u}{\partial  x} + a_2 \frac{\partial w}{\partial y}\frac{\partial u}{\partial y} +  w(x,y) a_{0}u(x,y)\right) dR = \int_R w f \, dA + \int_{\partial S_2} w \bar{y}  dS
$$

# Problem 4

In [112]:
"""
Given Values
"""
E = 3E7
t_bar = 0
t_bar_y = -20
v = 0.3

"""
Calculate the D matrix
"""

D = E/(1-v**2)*sp.Matrix([[1, v, 0], [v, 1, 0], [0, 0, (1-v)/2]])

print("D Matrix")
sp.pprint(D)

D Matrix
⎡32967032.967033   9890109.89010989         0        ⎤
⎢                                                    ⎥
⎢9890109.89010989  32967032.967033          0        ⎥
⎢                                                    ⎥
⎣       0                 0          11538461.5384615⎦


In [113]:
"""
Set up the Coordinate Matrix
In Example 9.2 we only needed one coordinate system since we were dealing with a single quadrillateral
Additionally the nodal numbers used in Example 9.2 are different than the ones I am using (I chose to use figure 9.19 for my two triangular elements)
"""

# Split the quadrilateral into two triangles
triangle_1 = sp.Matrix([
    [0, 1],  # Node 2
    [0, 0],  # Node 1
    [2, 0.5] # Node 3
])

triangle_2 = sp.Matrix([
    [0, 1],  # Node 2
    [2, 0.5],# Node 3
    [2, 1]   # Node 4
])

# Print the coordinates for verification
print("Triangle 1 Coordinates (Nodes 2, 1, 3):")
sp.pprint(triangle_1)

print("\nTriangle 2 Coordinates (Nodes 2, 3, 4):")
sp.pprint(triangle_2)


# Print the coordinates for verification
print("Triangle 1 Coordinates:")
sp.pprint(triangle_1)

print("\nTriangle 2 Coordinates:")
sp.pprint(triangle_2)

Triangle 1 Coordinates (Nodes 2, 1, 3):
⎡0   1 ⎤
⎢      ⎥
⎢0   0 ⎥
⎢      ⎥
⎣2  0.5⎦

Triangle 2 Coordinates (Nodes 2, 3, 4):
⎡0   1 ⎤
⎢      ⎥
⎢2  0.5⎥
⎢      ⎥
⎣2   1 ⎦
Triangle 1 Coordinates:
⎡0   1 ⎤
⎢      ⎥
⎢0   0 ⎥
⎢      ⎥
⎣2  0.5⎦

Triangle 2 Coordinates:
⎡0   1 ⎤
⎢      ⎥
⎢2  0.5⎥
⎢      ⎥
⎣2   1 ⎦


In [114]:
'''
Create the Shape Functions
'''
# triangular shape functions

xi, eta = sp.symbols('xi eta')

N1 = 1 - xi - eta 
N2 = xi
N3 = eta

In [115]:
'''
Compute the Jacobian Matrix
'''

# Shape function derivatives
dN1dxi = sp.diff(N1, xi)
dN2dxi = sp.diff(N2, xi)
dN3dxi = sp.diff(N3, xi)
dN1deta = sp.diff(N1, eta)
dN2deta = sp.diff(N2, eta)
dN3deta = sp.diff(N3, eta)

# Derivatives of shape functions with respect to xi and eta
dN_dxi = sp.Matrix([dN1dxi, dN2dxi, dN3dxi])   # [∂N1/∂xi, ∂N2/∂xi, ∂N3/∂xi]
dN_deta = sp.Matrix([dN1deta, dN2deta, dN3deta]) # [∂N1/∂eta, ∂N2/∂eta, ∂N3/∂eta]

# Combine derivatives into a single matrix
dN = sp.Matrix.hstack(dN_dxi, dN_deta)  # Shape function derivative matrix

# Compute the Jacobian matrix
J_triangle1 = dN.T * triangle_1

print("Jacobian Matrix for Triangle 1:")
sp.pprint(J_triangle1)

J_triangle2 = dN.T * triangle_2

print("\nJacobian Matrix for Triangle 2:")
sp.pprint(J_triangle2)

Jacobian Matrix for Triangle 1:
⎡0   -1 ⎤
⎢       ⎥
⎣2  -0.5⎦

Jacobian Matrix for Triangle 2:
⎡2  -0.5⎤
⎢       ⎥
⎣2   0  ⎦


In [116]:
'''
Determinant of the Jacobian Matrix
Confirm that the determinant is not equal to zero

Then I took the inverse of the Jacobian Matrix
'''
det_J_triangle1 = sp.det(J_triangle1)

print("Determinant of the Jacobian Matrix of triangle 1:")
sp.pprint(det_J_triangle1)

inv_J_triangle1 = J_triangle1.inv()

print("Inverse of the Jacobian Matrix of triangle 1:")
sp.pprint(inv_J_triangle1)

det_J_triangle2 = sp.det(J_triangle2)

print("Determinant of the Jacobian Matrix of triangle 2:")
sp.pprint(det_J_triangle2)

inv_J_triangle2 = J_triangle2.inv()

print("Inverse of the Jacobian Matrix of triangle 2:")
sp.pprint(inv_J_triangle2)

Determinant of the Jacobian Matrix of triangle 1:
2
Inverse of the Jacobian Matrix of triangle 1:
⎡-0.25  1/2⎤
⎢          ⎥
⎣ -1     0 ⎦
Determinant of the Jacobian Matrix of triangle 2:
1.00000000000000
Inverse of the Jacobian Matrix of triangle 2:
⎡ 0    0.5⎤
⎢         ⎥
⎣-2.0  2.0⎦


In [117]:
'''
Calculate the B matrix for both triangles
'''

# Derivatives of shape functions in parent coordinates
dN_parent = sp.Matrix([
    [-1, 1, 0],  # ∂N/∂xi
    [-1, 0, 1]   # ∂N/∂eta
])

# Compute derivatives in global coordinates
global_derivatives = inv_J_triangle1 * dN_parent

# Construct the B-matrix for Triangle 1
B1 = sp.zeros(3, 6)  # Initialize a 3x6 matrix

# Assign ∂N/∂x
B1[0, 0] = global_derivatives[0, 0]
B1[0, 2] = global_derivatives[0, 1]
B1[0, 4] = global_derivatives[0, 2]

# Assign ∂N/∂y
B1[1, 1] = global_derivatives[1, 0]
B1[1, 3] = global_derivatives[1, 1]
B1[1, 5] = global_derivatives[1, 2]

# Assign ∂N/∂y and ∂N/∂x for the shear terms
B1[2, 0] = global_derivatives[1, 0]
B1[2, 2] = global_derivatives[1, 1]
B1[2, 4] = global_derivatives[1, 2]
B1[2, 1] = global_derivatives[0, 0]
B1[2, 3] = global_derivatives[0, 1]
B1[2, 5] = global_derivatives[0, 2]


print("Strain-Displacement Matrix (B) for Triangle 1:")
sp.pprint(B1)


# Compute derivatives in global coordinates for Triangle 2
global_derivatives2 = inv_J_triangle2 * dN_parent

# Construct the B-matrix for Triangle 2
B2 = sp.zeros(3, 6)  # Initialize a 3x6 matrix

# Assign ∂N/∂x
B2[0, 0] = global_derivatives2[0, 0]
B2[0, 2] = global_derivatives2[0, 1]
B2[0, 4] = global_derivatives2[0, 2]

# Assign ∂N/∂y
B2[1, 1] = global_derivatives2[1, 0]
B2[1, 3] = global_derivatives2[1, 1]
B2[1, 5] = global_derivatives2[1, 2]

# Assign ∂N/∂y and ∂N/∂x for the shear terms
B2[2, 0] = global_derivatives2[1, 0]
B2[2, 2] = global_derivatives2[1, 1]
B2[2, 4] = global_derivatives2[1, 2]
B2[2, 1] = global_derivatives2[0, 0]
B2[2, 3] = global_derivatives2[0, 1]
B2[2, 5] = global_derivatives2[0, 2]

print("\nStrain-Displacement Matrix (B) for Triangle 2:")
sp.pprint(B2)


Strain-Displacement Matrix (B) for Triangle 1:
⎡-0.25    0    -0.25    0    1/2   0 ⎤
⎢                                    ⎥
⎢  0      1      0     -1     0    0 ⎥
⎢                                    ⎥
⎣  1    -0.25   -1    -0.25   0   1/2⎦

Strain-Displacement Matrix (B) for Triangle 2:
⎡-0.5   0     0     0    0.5   0 ⎤
⎢                                ⎥
⎢ 0     0     0    -2.0   0   2.0⎥
⎢                                ⎥
⎣ 0    -0.5  -2.0   0    2.0  0.5⎦


In [118]:
'''
Calculate the global stiffness matrix
'''

# Define the Gauss quadrature integration weight and point
w = 1  # Weight for a single integration point
xi, eta = 1/3, 1/3  # Barycentric coordinates for the integration point

# Element stiffness matrix for Triangle 1
K1 = (B1.T * D * B1) * det_J_triangle1 * w
print("\nElement Stiffness Matrix for Triangle 1:")
sp.pprint(K1)

# Element stiffness matrix for Triangle 2
K2 = (B2.T * D * B2) * det_J_triangle2 * w
print("\nElement Stiffness Matrix for Triangle 2:")
sp.pprint(K2)


# Initialize the global stiffness matrix (6 DOF for 4 nodes)
K_global = sp.zeros(8, 8)

# Assembly for Triangle 1 (Nodes 2, 1, 3 -> Global DOFs [2, 3, 0, 1, 4, 5])
node_map1 = [2, 3, 0, 1, 4, 5]
for i in range(6):
    for j in range(6):
        K_global[node_map1[i], node_map1[j]] += K1[i, j]

# Assembly for Triangle 2 (Nodes 2, 3, 4 -> Global DOFs [2, 3, 4, 5, 6, 7])
node_map2 = [2, 3, 4, 5, 6, 7]
for i in range(6):
    for j in range(6):
        K_global[node_map2[i], node_map2[j]] += K2[i, j]

print("\nGlobal Stiffness Matrix:")
sp.pprint(K_global)


Element Stiffness Matrix for Triangle 1:
⎡27197802.1978022   -10714285.7142857  -18956043.956044   -824175.824175824  -
⎢                                                                             
⎢-10714285.7142857  67376373.6263736   824175.824175824   -64491758.2417582  9
⎢                                                                             
⎢-18956043.956044   824175.824175824   27197802.1978022   10714285.7142857   -
⎢                                                                             
⎢-824175.824175824  -64491758.2417582  10714285.7142857   67376373.6263736   -
⎢                                                                             
⎢-8241758.24175824  9890109.89010989   -8241758.24175824  -9890109.89010989  1
⎢                                                                             
⎣11538461.5384615   -2884615.38461538  -11538461.5384615  -2884615.38461538   

8241758.24175824  11538461.5384615 ⎤
                                   ⎥
890109.8901098

In [119]:
'''
Apply the boundary conditions
'''
# Initialize the global force vector (8 DOFs for 4 nodes)
F_global = sp.zeros(8, 1)

# Apply the boundary conditions
constrained_dofs = [0, 1]  # DOFs for Node 1 (fixed at x=0, y=0)

# Modify the global stiffness matrix and force vector for the constrained DOFs
for dof in constrained_dofs:
    K_global[dof, :] = sp.zeros(1, K_global.shape[1])  # Zero out the entire row
    K_global[:, dof] = sp.zeros(K_global.shape[0], 1)  # Zero out the entire column
    K_global[dof, dof] = 1  # Set diagonal to 1 to maintain structure
    F_global[dof] = 0  # Set force at constrained DOF to zero

# Print the modified global stiffness matrix and force vector
print("\nModified Global Stiffness Matrix:")
sp.pprint(K_global)

print("\nModified Global Force Vector:")
sp.pprint(F_global)

# Define edge length and traction force
t_y = -20  # Traction in the y-direction
L_edge = sp.sqrt((2 - 2)**2 + (1 - 0.5)**2)  # Length of edge between Node 3 and Node 4 (0.5)

# Shape functions along the edge
eta = sp.symbols('eta')  # Natural coordinate along the edge
N3_edge = (1 - eta) / 2  # Shape function for Node 3
N4_edge = (1 + eta) / 2  # Shape function for Node 4

# Integrate the force contributions
F3 = sp.integrate(N3_edge * t_y * L_edge / 2, (eta, -1, 1))
F4 = sp.integrate(N4_edge * t_y * L_edge / 2, (eta, -1, 1))

# Initialize the global force vector (8 DOFs for 4 nodes)
F_global = sp.zeros(8, 1)

# Assign contributions to the global force vector
F_global[4] += F3  # y-direction DOF of Node 3
F_global[6] += F4  # y-direction DOF of Node 4

# Print the updated global force vector
print("\nUpdated Global Force Vector with Traction Contribution:")
sp.pprint(F_global)


Modified Global Stiffness Matrix:
⎡1  0          0                  0                  0                  0     
⎢                                                                             
⎢0  1          0                  0                  0                  0     
⎢                                                                             
⎢0  0  35439560.4395604   -10714285.7142857  -8241758.24175824  21428571.42857
⎢                                                                             
⎢0  0  -10714285.7142857   70260989.010989   21428571.4285714   -2884615.38461
⎢                                                                             
⎢0  0  -8241758.24175824  21428571.4285714   62637362.6373626           0     
⎢                                                                             
⎢0  0  21428571.4285714   -2884615.38461538          0          137637362.6373
⎢                                                                             
⎢0  0  -8241758.2

In [120]:
'''
Since xi is fixed for the traction force and nodes 3 and 4 are on the same edge, I can simplify the matrices
So I set up a reduced matrix for the k AND d values
'''

# Extract the bottom-right 4x4 section
K_reduced = K_global[-4:, -4:]

# Print the extracted matrix
print("\nReduced Global Stiffness Matrix (K_reduced):")
sp.pprint(K_reduced)

dy = sp.Matrix([sp.symbols('ux3'), sp.symbols('uy3'), sp.symbols('ux4'), sp.symbols('uy4')])

f_reduced = sp.Matrix([-5,0,-5,0])



Reduced Global Stiffness Matrix (K_reduced):
⎡62637362.6373626           0          -46153846.1538462  -11538461.5384615⎤
⎢                                                                          ⎥
⎢        0          137637362.637363   -9890109.89010989  -131868131.868132⎥
⎢                                                                          ⎥
⎢-46153846.1538462  -9890109.89010989  54395604.3956044   21428571.4285714 ⎥
⎢                                                                          ⎥
⎣-11538461.5384615  -131868131.868132  21428571.4285714   134752747.252747 ⎦


In [121]:
'''
Solve the reduced system
'''

d_reduced = K_reduced.LUsolve(f_reduced)

print("\nDisplacements for Unconstrained Nodes:")
sp.pprint(d_reduced)



Displacements for Unconstrained Nodes:
⎡-3.99197710425073e-7⎤
⎢                    ⎥
⎢ 6.8474555065528e-8 ⎥
⎢                    ⎥
⎢-4.59926183103437e-7⎥
⎢                    ⎥
⎣1.05964780582403e-7 ⎦


In [122]:
'''
Calculate the resulting strains and stresses
'''

# Extract displacements for Triangle 1 (Nodes 1, 2, 3 -> DOFs [0, 1, 2, 3, 4, 5])
d1 = sp.Matrix([
    0, 0,  # Node 1 (fixed)
    d_reduced[0], d_reduced[1],  # Node 2
    d_reduced[2], d_reduced[3]   # Node 3
])

# Compute strain for Triangle 1
strain_1 = B1 * d1
print("\nStrain for Triangle 1:")
sp.pprint(strain_1)

# Compute stress for Triangle 1
stress_1 = D * strain_1
print("\nStress for Triangle 1:")
sp.pprint(stress_1)

# Extract displacements for Triangle 2 (Nodes 2, 3, 4 -> DOFs [2, 3, 4, 5, 6, 7])
d2 = sp.Matrix([
    d_reduced[0], d_reduced[1],  # Node 2
    d_reduced[2], d_reduced[3],  # Node 3
    0, 0  # Node 4 (fixed)
])

# Compute strain for Triangle 2
strain_2 = B2 * d2
print("\nStrain for Triangle 2:")
sp.pprint(strain_2)

# Compute stress for Triangle 2
stress_2 = D * strain_2
print("\nStress for Triangle 2:")
sp.pprint(stress_2)



Strain for Triangle 1:
⎡-1.3016366394545e-7⎤
⎢                   ⎥
⎢-6.8474555065528e-8⎥
⎢                   ⎥
⎣4.35061461949892e-7⎦

Stress for Triangle 1:
⎡-4.96833067467392⎤
⎢                 ⎥
⎢-3.54473585436802⎥
⎢                 ⎥
⎣5.01993994557568 ⎦

Strain for Triangle 2:
⎡1.99598855212536e-7 ⎤
⎢                    ⎥
⎢-2.11929561164805e-7⎥
⎢                    ⎥
⎣8.85615088674111e-7 ⎦

Stress for Triangle 2:
⎡4.48417539109104 ⎤
⎢                 ⎥
⎢-5.01263421761685⎥
⎢                 ⎥
⎣10.2186356385474 ⎦


In [ ]:
'''
Code to get the stress and strain values for each node separately
'''

# Gauss Point for 1-point integration
xi_eta_points = [(1/3, 1/3)]  # You can add more points for higher-order integration
weights = [1]  # Corresponding weights

# Loop over Gauss points for Triangle 1
print("\nStrains and Stresses for Triangle 1:")
for (xi, eta), w in zip(xi_eta_points, weights):
    # Evaluate the Jacobian and its determinant at this Gauss point
    J_eval = J_triangle1.subs({'xi': xi, 'eta': eta})
    det_J_eval = det_J_triangle1.subs({'xi': xi, 'eta': eta})
    inv_J_eval = J_eval.inv()

    # Compute derivatives in global coordinates
    global_derivatives = inv_J_eval * dN_parent

    # Construct the B-matrix for this Gauss point
    B_gauss = sp.zeros(3, 6)
    B_gauss[0, 0] = global_derivatives[0, 0]
    B_gauss[0, 2] = global_derivatives[0, 1]
    B_gauss[0, 4] = global_derivatives[0, 2]
    B_gauss[1, 1] = global_derivatives[1, 0]
    B_gauss[1, 3] = global_derivatives[1, 1]
    B_gauss[1, 5] = global_derivatives[1, 2]
    B_gauss[2, 0] = global_derivatives[1, 0]
    B_gauss[2, 2] = global_derivatives[1, 1]
    B_gauss[2, 4] = global_derivatives[1, 2]
    B_gauss[2, 1] = global_derivatives[0, 0]
    B_gauss[2, 3] = global_derivatives[0, 1]
    B_gauss[2, 5] = global_derivatives[0, 2]

    # Compute strain at this Gauss point
    strain_gauss = B_gauss * d1
    print(f"\nStrain at Gauss Point ({xi}, {eta}):")
    sp.pprint(strain_gauss)

    # Compute stress at this Gauss point
    stress_gauss = D * strain_gauss
    print(f"\nStress at Gauss Point ({xi}, {eta}):")
    sp.pprint(stress_gauss)

# Repeat for Triangle 2
print("\nStrains and Stresses for Triangle 2:")
for (xi, eta), w in zip(xi_eta_points, weights):
    # Evaluate the Jacobian and its determinant at this Gauss point
    J_eval = J_triangle2.subs({'xi': xi, 'eta': eta})
    det_J_eval = det_J_triangle2.subs({'xi': xi, 'eta': eta})
    inv_J_eval = J_eval.inv()

    # Compute derivatives in global coordinates
    global_derivatives = inv_J_eval * dN_parent

    # Construct the B-matrix for this Gauss point
    B_gauss = sp.zeros(3, 6)
    B_gauss[0, 0] = global_derivatives[0, 0]
    B_gauss[0, 2] = global_derivatives[0, 1]
    B_gauss[0, 4] = global_derivatives[0, 2]
    B_gauss[1, 1] = global_derivatives[1, 0]
    B_gauss[1, 3] = global_derivatives[1, 1]
    B_gauss[1, 5] = global_derivatives[1, 2]
    B_gauss[2, 0] = global_derivatives[1, 0]
    B_gauss[2, 2] = global_derivatives[1, 1]
    B_gauss[2, 4] = global_derivatives[1, 2]
    B_gauss[2, 1] = global_derivatives[0, 0]
    B_gauss[2, 3] = global_derivatives[0, 1]
    B_gauss[2, 5] = global_derivatives[0, 2]

    # Compute strain at this Gauss point
    strain_gauss = B_gauss * d2
    print(f"\nStrain at Gauss Point ({xi}, {eta}):")
    sp.pprint(strain_gauss)

    # Compute stress at this Gauss point
    stress_gauss = D * strain_gauss
    print(f"\nStress at Gauss Point ({xi}, {eta}):")
    sp.pprint(stress_gauss)


Strains and Stresses for Triangle 1:

Strain at Gauss Point (0.3333333333333333, 0.3333333333333333):
⎡-1.3016366394545e-7⎤
⎢                   ⎥
⎢-6.8474555065528e-8⎥
⎢                   ⎥
⎣4.35061461949892e-7⎦

Stress at Gauss Point (0.3333333333333333, 0.3333333333333333):
⎡-4.96833067467392⎤
⎢                 ⎥
⎢-3.54473585436802⎥
⎢                 ⎥
⎣5.01993994557568 ⎦

Strains and Stresses for Triangle 2:

Strain at Gauss Point (0.3333333333333333, 0.3333333333333333):
⎡1.99598855212536e-7 ⎤
⎢                    ⎥
⎢-2.11929561164805e-7⎥
⎢                    ⎥
⎣8.85615088674111e-7 ⎦

Stress at Gauss Point (0.3333333333333333, 0.3333333333333333):
⎡4.48417539109104 ⎤
⎢                 ⎥
⎢-5.01263421761685⎥
⎢                 ⎥
⎣10.2186356385474 ⎦
